在 chat 中维护上下文

简单的讲所有聊天记录都传给 llm 很容易受到 llm 的上下文窗口限制，也会消耗大量 token  
并且用户后续发送的信息可能与前面聊天讨论的话题完全无关，可能会影响回答的质量

因此我们可以在聊天记录进行一些处理，这就是 memory

In [1]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";

const history = new ChatMessageHistory();

// 添加 Message 历史信息
await history.addMessage(new HumanMessage('hi'));
await history.addMessage(new AIMessage("What can I do for you?"))

const message = await history.getMessages();

console.log(message);

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "hi", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What can I do for you?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What can I do for you?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]
